In [1]:
import re
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Вспомогательные функции
def check_for_none(x):
    if x is None:
        return ''
    else:
        return x.text

# Parsing

## Avito

In [3]:
main_url = 'https://www.avito.ru'

In [8]:
final_data = []

for i in range(1,11):
    add_url = '/moskva_i_mo/avtomobili?p='+str(i)
    pred_ap = {}
    
    print(i)
    # Короткий текст объявления в шаблоне поиска
    url = main_url + add_url
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    ap = soup.findAll('div', {'class': 'snippet-horizontal'})
    for a in ap:  
        time.sleep(2)
        pred_ap['main_title'] = check_for_none(a.find('div', {'class': 'snippet-title-row'}))
        pred_ap['price_comment'] = check_for_none(a.find('span', {'class': 'snippet-tag'}))
        pred_ap['price'] = check_for_none(a.find('span', {'class': 'snippet-price'}))
        pred_ap['specific-params'] = check_for_none(a.find('div', {'class': 'specific-params_block'}))
        pred_ap['address'] = check_for_none(a.find('div', {'class': 'item-address'}))

        ########################################################################
        # Полный текст внутри объявления
        full_app_dict = {}

        url = main_url + a.find('a', {'class': 'snippet-link'}, href=True)['href']
        r = requests.get(url)
        soup = BeautifulSoup(r.text)

        print(url)
        fap = soup.find('div', {'class': 'item-params'})
        li = fap.findAll('li', {'class':'item-params-list-item'})
        for l in li:
            full_app_dict[l.text.split(':')[0].strip()] = l.text.split(':')[1].strip()
        full_app_dict['address'] = soup.find('div', {'class':'item-address'}).text
        full_app_dict['full_text_app'] = soup.find('div', class_=re.compile('item-description')).text

        ########################################################################
        # Дополенная информация об объекте
        adv_info = {}

        fap_adv = soup.find('ul', class_=re.compile('advanced-params'))
        if fap_adv is not None:
            li = fap_adv.findAll('li', {'class': 'advanced-params-param'})
            for l in li:
                param_list = []
                title = l.find('div', {'class': 'advanced-params-param-title'}).text
                item_list = l.find('ul', {'class': 'advanced-params-param-list'})  
                for il in item_list.findAll('li', {'class': 'advanced-params-param-item'}):
                        param_list.append(il.text)    
                adv_info[title] = param_list

        ########################################################################
        #Дополнительная информация по цене в объявлении  
        add_price = {}

        js = soup.find('div', class_ = re.compile('js-ssr'))
        if js is not None:    
            add_info = json.loads(js['data-props'])
            add_price['title'] = add_info['title']
            add_price['subtitle'] = add_info['subtitle']
            add_price['priceType'] = add_info['priceType']
            add_price['marketPrice'] = add_info['marketPrice']
            add_price['isVisibleForUser'] = add_info['isVisibleForUser']
            add_price['price'] = add_info['price']
        else:
            add_price['title'] = ''
            add_price['subtitle'] = ''
            add_price['priceType'] = ''
            add_price['marketPrice'] = ''
            add_price['isVisibleForUser'] = ''
            add_price['price'] = ''

        ########################################################################
        # Технические характеритики модели
        spec_table = {}
        url = main_url + soup.find('a', {'class': 'item-view-specification-link'}, href=True)['href']
        #print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.text)

        table = soup.findAll('table', class_=re.compile('specification-table'))
        for tab in table:
            tr = tab.findAll('tr')
            for t in tr:
                td = t.findAll('td')
                spec_table[td[0].text] = td[1].text

        # Собираем все данныe в один DataFrame
        final_data.append({**spec_table, **add_price, **pred_ap, **adv_info, **full_app_dict})

1
200
<!DOCTYPE html>
<html> <head>
<script>
 window.dataLayer = [{"userAuth":0,"pageType":"Other"}];
 (function(w, d, s, l, i) {
 w[l] = w[l] || [];
 w[l].push({
 'gtm.start': new Date().getTime(),
 event: 'gtm.js'
 });
 var f = d.getElementsByTagName(s)[0],
 j = d.createElement(s),
 dl = l != 'dataLayer' ? '&l=' + l : '';
 j.async = true;
 j.src = '//www.googletagmanager.com/gtm.js?id=' + i + dl;
 f.parentNode.insertBefore(j, f);
 })(window, document, 'script', 'dataLayer', 'GTM-KP9Q9H');
 </script>
<meta charset="utf-8"/> <title>Доступ временно заблокирован</title>
<link href="https://static.avito.ru/s/cc/styles/550b2d3f77e52ae0fa35.css" rel="stylesheet"/>
<script src="https://static.avito.ru/s/cc/bundles/9e363b6196043b3fa9c3.js"></script>
<link href="https://static.avito.ru/s/cc/styles/7d6753a600db7f4f5513.css" rel="stylesheet"/>
<script src="https://static.avito.ru/s/cc/bundles/f84d02d4e3d5cd0d9870.js"></script>
</head> <body>
<noscript> <iframe height="0" src="//www.googletagmana

In [ ]:
avito_data = pd.DataFrame(final_data)
avito_data.head()